### Assignment 2

## Written Assignment

1. Read [Deep Learning: An Introduction for Applied Mathematicians](https://epubs.siam.org/doi/10.1137/18M1165748).
   Consider a network as defined in (3.1) and (3.2). Assume that $n_L=1$, find an algorithm to calculate $\nabla a^{[L]}(x)$.
   
<div align="center">
    
$$
\begin{array}{cl}
1 & x^{\{k\}} \text{ is the current training data point.} \\[10pt]
2 & a^{[1]} = x^{\{k\}} \\[10pt]
3 & \text{For } l=2 \text{ upto } L \\[10pt]
4 & \quad z^{[l]} = W^{[l]}a^{[l-1]} + b^{[l]} \\[10pt]
5 & \quad a^{[l]} = \sigma(z^{[l]}) \\[10pt]
6 & \quad D^{[l]} = \text{diag}(\sigma'(z^{[l]})) \\[10pt]
7 & \text{end} \\[10pt]
8 & \text{\# Define } \nabla a^{[L]} = \begin{pmatrix} \dfrac{\partial a^{[L]}}{\partial w_{11}} & \dfrac{\partial a^{[L]}}{\partial w_{21}} & \cdots & \dfrac{\partial a^{[L]}}{\partial w_{n_l 1}} & \dfrac{\partial a^{[L]}}{\partial b} \end{pmatrix}^T \\[10pt]
9 & \text{For } i=1 \text{ upto } n_{l}-1 \\[10pt]
10 & \quad a_i = \sigma'(z_i^{[L]})a^{[l-1]} \\[10pt]
11 & \text{end} \\[10pt]
12 & a_{n_l} = \sigma'(z_j^{[L]})
\end{array}
$$
    
</div>

<div style="background-color: black; color: white; border: 1px solid #444444; border-radius: 5px; padding: 15px 20px; margin: 15px 0;">
    <b><i>Proof.</i></b>
    <br><br>
    <br><br>
    First, observe that $\dfrac{\partial a_j^{[l]}}{\partial z_j}=\dfrac{\partial\sigma(z_j^{[L]})}{\partial z_j}=\sigma'(z_j^{[L]})$

$$
\begin{array}{rcl}
\dfrac{\partial a_j^{[L]}}{\partial w_{mn}}&=&\dfrac{\partial a_j}{\partial z_j}\cdot\dfrac{\partial z_j}{\partial w_{mn}}\\[10pt]
&=&\sigma'(z_j^{[L]})\dfrac{\partial}{\partial w_{mn}}\displaystyle\sum_{n=1}^{n_{L-1}}\left(w_{jn}a_n^{[L-1]}+b_j^{[L]}\right)\\[10pt]
&=&\sigma'(z_j^{[L]})a_n^{[L-1]}\\[10pt]
\dfrac{\partial a_j^{[L]}}{\partial b_j}&=&\sigma'(z_j^{[L]})
\end{array}
$$
    <span style="float: right;">□</span>
</div>

2. I was wondering how to fix the problem(the oscillation on the both sides) in Figure 10 below. I've tried to change the way I select the data but it doesn't work.

<hr style="border-style: dashed; border-color: white; border-width: 0.8px;">

## Programming Assignment
We are asked to approximate $f(x)=\dfrac{1}{1+25x^2}$ in $[-1,1]$ and write a short report.

I think that there might be something strange if I just describe it briefly. 

Thus, a summary of each approach is provided below. For a more detailed process, please refer to the corresponding notebooks.

# Approximate by polynomials
I have tried to change almost every hyperparameters and the results are really bad, the model doesn't learn well and it can't add layers in the hidden part because we can increase the degree of the polynomial to have the same effect. 

```text
--- Final Result ---
Final MSE (Degree=24): 0.019440
Final Max Error: 0.360167
```

<figure id="Polynomial 1 Epoch:40000 Datanum:1001">
    <img src="Polynomial_24_0.01_40000_100001.png" alt="Polynomial 1 Layer" style="width: 70%;">
    <figcaption><b>Figure 6</b>: Polynomial Degree:24 Epoch:40000 Datanum:100001.</figcaption>
</figure>

I have tried a lot in adjusting the hyperparameters but it was invalid; the MSE remained at $10^{-2}$ and the Max Error also remained at $10^{-1}$, which showed that the failure of the model didn't depends on the insufficiency of the training rounds or the data.

I deduced that the main reason is the defect of the polynomial model.

Thus, I try another method.

# Approximate by Trigonometric Functions
The way I understand the first part is something like Taylor Series, which is reminiscent of Fourier Series.

In the other words, I just try to use Trigonometric Functions to deal with this problem.

Before I start, note that the Runge function is an even function:

$$
f(-x)=\dfrac{1}{1+25(-x)^2}=\dfrac{1}{1+25x^2}=f(x)
$$

Thus, I just use the cosine functions as my hypothesis.

However, it had a horrible beggining.

```text
--- Final Result ---
Final MSE: 0.014785
Final Max Error: 0.311562
```
<figure id="Fourier Learning rate:0.01 Layer:1 Epoch:20000 Datanum:1001">
    <img src="Fourier_0.01_20000_1001.png" alt="Fourier 1 Layer" style="width: 70%;">
    <figcaption><b>Figure 7</b>: Fourier Learning rate:0.01 Layer:1 Epoch:20000 Datanum:1001.</figcaption>
</figure>

I found that the shape did not match my expectations as well as I had imagined, and I have tried to add more data, but it wasn't effective.

The result did not improve significantly until I add layers in the hidden part, and the middle part almost match the shape of the target, nevertheless, it oscillates severely on both sides, especially for the lowest learning rate, which is shown in Figure 12.

```text
--- Final Result ---
Final MSE: 0.000074
Final Max Error: 0.022620
```

<figure id="Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10001">
    <img src="Fourier_0.01_20000_10001_3.png" alt="Fourier 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 10</b>: Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10001.</figcaption>
</figure>

I found that the learning rate I had set was too large for this model, and it made the gradient "explode". 

Thus, I decrease the learning rate.
```text
--- Final Result ---
Final MSE: 0.000265
Final Max Error: 0.076985
```

<figure id="Fourier Learning rate:0.001 Layer:3 Epoch:20000 Datanum:10001">
    <img src="Fourier_0.001_20000_10001_3.png" alt="Fourier 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 11</b>: Fourier Learning rate:0.001 Layer:3 Epoch:20000 Datanum:10001.</figcaption>
</figure>

```text
--- Final Result ---
Final MSE: 0.003896
Final Max Error: 0.226449
```
<figure id="Fourier Learning rate:0.0001 Layer:3 Epoch:20000 Datanum:10001">
    <img src="Fourier_0.0001_20000_10001_3.png" alt="Fourier 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 12</b>: Fourier Learning rate:0.0001 Layer:3 Epoch:20000 Datanum:10001.</figcaption>
</figure>

After adjusting the hyperparameters, I find that even if the graph in Figure 10 almost matches the function we want to approach, but we can also see the oscillations on both sides.

```text
--- Final Result ---
Final MSE: 0.000074
Final Max Error: 0.022620
```

<figure id="Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10001">
    <img src="Fourier_0.01_20000_10001_3.png" alt="Fourier 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 10</b>: Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10001.</figcaption>
</figure>

I consider this to be a form of overfitting, however, a better performence in the middle part is undeniable.

Therefore, I try to adjust the way I select the data.

```python
x_center = jnp.linspace(-0.5, 0.5, datanum//2)
x_left = jnp.linspace(-1.0, -0.5, datanum//4)
x_right = jnp.linspace(0.5, 1.0, datanum//4)
x_train = jnp.concatenate([x_center, x_left, x_right])
```

```text
--- Final Result ---
Final MSE: 0.001097
Final Max Error: 0.093484
```
<figure id="Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10000">
    <img src="Fourier_0.01_20000_10000_3_s.png" alt="Fourier 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 10'</b>: Fourier Learning rate:0.01 Layer:3 Epoch:20000 Datanum:10000.</figcaption>
</figure>

However, it did not work, also the oscillation on both sides still existed.

Then, I try another function we have learned in class, the sigmoid function.


# Approximate by Sigmoid Function
At first, I start it with a single layer in the hidden part, and its results are better than the others.

```text
--- Final Result ---
Final MSE: 0.000715
Final Max Error: 0.074811
```
<figure id="Sigmoid Learning rate:0.01 Layer:1 Epoch:20000 Datanum:1001">
    <img src="Sigmoid_0.01_20000_1001.png" alt="Sigmoid 1 Layer" style="width: 70%;">
    <figcaption><b>Figure 15</b>: Sigmoid Learning rate:0.01 Layer:1 Epoch:20000 Datanum:1001.</figcaption>
</figure>

```text
--- Final Result ---
Final MSE: 0.000003
Final Max Error: 0.004661
```
<figure id="Sigmoid Learning rate:0.01 Layer:1 Epoch:20000 Datanum:10001">
    <img src="Sigmoid_0.01_20000_10001.png" alt="Sigmoid 1 Layer" style="width: 70%;">
    <figcaption><b>Figure 16</b>: Sigmoid Learning rate:0.01 Layer:1 Epoch:20000 Datanum:10001.</figcaption>
</figure>


The most fascinating part is even if I just tried to enhance the number of data, the MSE dip to $0.000003$.

In other words, even before deepening the network, providing sufficient data dramatically improved the model's performance.

Then, I tried to add more layers to the hidden part.

The program below is the revised version, which adds more layers.

We can find that the middle part has a gap but the "awful oscillation" didn't appear again.

After this, I try to add some layers in the hidden part, and we can get this beautiful result.

```text
--- Final Result ---
Final MSE: 0.000001
Final Max Error: 0.002345
```
<figure id="Sigmoid Learning rate:0.01 Layer:3 Epoch:40000 Datanum:10001">
    <img src="Sigmoid_0.01_40000_10001_3.png" alt="Sigmoid 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 20</b>: Sigmoid Learning rate:0.01 Layer:3 Epoch:40000 Datanum:10001.</figcaption>
</figure>

We can find that it perfectly match the true graph, and the loss decreased to $10^{-6}$. 

Thus, I decided to use it to take the extra task, compute the derivative, and the results are presented below.

* Extra: Compute the Derivative

```text
--- Final Result ---
Final MSE: 0.000001
Final Max Error: 0.002345
--- Final Result ---
Final MSE(Derivative): 0.000435
Final Max Error(Derivative): 0.073273
```
<figure id="Sigmoid Learning rate:0.01 Layer:3 Epoch:40000 Datanum:10001">
    <img src="Sigmoid_0.01_40000_10001_3_d.png" alt="Sigmoid 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 20'</b>: Based on 20.</figcaption>
</figure>
<figure id="Sigmoid Learning rate:0.01 Layer:3 Epoch:40000 Datanum:10001">
    <img src="Sigmoid_0.01_40000_10001_3_d'.png" alt="Sigmoid 3 Layers" style="width: 70%;">
    <figcaption><b>Figure 21</b>: Compute Derivative.</figcaption>
</figure>

<hr style="border-style: dashed; border-color: white; border-width: 2px;">

# Program

```python
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

jax.config.update("jax_enable_x64", True)

# The Runge Function
def f(x):
    return 1/(1+25*x**2)

# Hyperparameters
learning_rate = 0.01
epochs = 40000
datanum = 10001
batch_size = 32 # Mini-Batch (I only use it in the Sigmoid model.)

# Data
x_train = jnp.linspace(-1.0, 1.0, datanum)
y_train = f(x_train)

key = jax.random.PRNGKey(0)
key, w1_key, b1_key, w2_key, b2_key, w3_key, b3_key = jax.random.split(key, 7)
params = {
    'w1': jax.random.normal(w1_key, (1, 16)), 'b1': jax.random.normal(b1_key, (16,)),
    'w2': jax.random.normal(w2_key, (16, 16)), 'b2': jax.random.normal(b2_key, (16,)),
    'w3': jax.random.normal(w3_key, (16, 1)), 'b3': jax.random.normal(b3_key, (1,))
}

# Construct the Sigmoid model
def deep_sigmoid_model(params, x):
    x = x.reshape(-1, 1)
    hidden1 = jax.nn.sigmoid(x @ params['w1'] + params['b1'])
    hidden2 = jax.nn.sigmoid(hidden1 @ params['w2'] + params['b2'])
    output = hidden2 @ params['w3'] + params['b3']
    return output

def loss_fn(params, x, y):
    predictions = deep_sigmoid_model(params, x).squeeze()
    return jnp.mean((predictions - y)**2)
loss_history = []

@jax.jit
# Gradient Descent
def update_step(params, x, y, learning_rate):
    grads = jax.grad(loss_fn)(params, x, y)
    return jax.tree_util.tree_map(lambda p, g: p - learning_rate * g, params, grads)


num_train = len(x_train)
steps_per_epoch = num_train // batch_size

key, shuffle_key = jax.random.split(key)
for epoch in range(epochs):
    shuffle_key, perm_key = jax.random.split(shuffle_key)  # Shuffle(I only use it in the Sigmoid model.)
    perm = jax.random.permutation(perm_key, num_train)

    for step in range(steps_per_epoch):
        batch_idx = perm[step * batch_size : (step + 1) * batch_size]
        x_batch, y_batch = x_train[batch_idx], y_train[batch_idx]
        params = update_step(params, x_batch, y_batch, learning_rate)

    if epoch % 1000 == 0:
        loss = loss_fn(params, x_train, y_train)
        loss_history.append(loss)
        print(f"Epoch {epoch}, Loss: {loss:.6f}")

x_plot = jnp.linspace(-1, 1, 500)
y_true = f(x_plot)
y_pred = deep_sigmoid_model(params, x_plot).squeeze()
y_pred_train = deep_sigmoid_model(params, x_train).squeeze()
final_mse = loss_fn(params, x_train, y_train)
max_error = jnp.max(jnp.abs(y_pred_train - y_train))

# Result
final_mse = loss_fn(params, x_train, y_train)
print(f"\n--- Final Result ---")
print(f"Final MSE: {final_mse:.6f}")
print(f"Final Max Error: {max_error:.6f}")

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(x_plot, y_true, label='True Runge Function')
plt.plot(x_plot, y_pred, label='Deep Sigmoid NN', linestyle='--')
plt.title('Deep Sigmoid Network Approximation')
plt.legend(); plt.grid(True)
plt.subplot(1, 2, 2)
plt.plot(range(0, epochs, 1000), loss_history)
plt.title('Training Loss Curve'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.yscale('log'); plt.grid(True)
plt.tight_layout()
plt.show()

# Derivative
def model_for_grad(x, model_params):
    return deep_sigmoid_model(model_params, x).squeeze()

grad_f = jax.grad(f)
y_d_true = jax.vmap(grad_f)(x_plot)
model_derivative_fn = jax.grad(model_for_grad, argnums=0)
vectorized_model_derivative = jax.vmap(model_derivative_fn, in_axes=(0, None))
y_d_pred = vectorized_model_derivative(x_plot, params)
pred_derivative_on_train = vectorized_model_derivative(x_train, params)
true_derivative_on_train = jax.vmap(grad_f)(x_train)
final_derivative_mse = jnp.mean((pred_derivative_on_train - true_derivative_on_train)**2)
max_derivative_error = jnp.max(jnp.abs(y_d_pred-y_d_true))
print(f"\n--- Final Result ---")
print(f"Final MSE(Derivative): {final_derivative_mse:.6f}")
print(f"Final Max Error(Derivative): {max_derivative_error:.6f}")

plt.figure(figsize=(8,6))
plt.plot(x_plot, y_d_true, label='True Derivative of Runge Function')
plt.plot(x_plot, y_d_pred, label='Deep Sigmoid NN', linestyle='--')
plt.title("Comparison of Derivatives")
plt.xlabel("x")
plt.ylabel("f'(x)")
plt.legend()
plt.grid(True)
plt.show()
```